# A naive look: GPU accelerated geodesic tracing
The work here is the research into first learning the [CUDA.jl](https://juliagpu.gitlab.io/CUDA.jl/) library and applying it to my Black Hole rendering project.

In [1]:
using CUDA, BenchmarkTools

## CPU Function
Here is a modified version of the function I am aiming to run on the GPU:

In [2]:
function truncator(px) 
    floor(min(max(0, px), 255))
end

function renderdisk(
    α::Float64,
    β::Float64,
    geodesics,
    ;
    height::Int=720,
    width::Int=1080,
    fov_index::Int=200,
    rinit::Function=zeros
    )

    data = rinit(Float64, (height, width, 3))

    mid = width ÷ 2 # integer division
    mid2 = height ÷ 2 # integer division
    n = length(geodesics)

    for x in (-mid):(mid - 1)
        
        col = rinit(Float64, (height, 3))
        
        for y in 1:height-1
            β2 = atan(y-mid2, x) # atan2
            r = sqrt(x^2 + (y-mid2)^2) # distance from middle of image
            i = convert(
                Int,
                floor(1 + r / fov_index * n)
            )
            if i < n
                col[height-y, :] .= intersection(geodesics[i], α, β + β2)
            end
        end

        data[:, x + mid + 1, :] .= truncator.(col)
    end

    sum(data)
end

renderdisk (generic function with 1 method)

For now, I'm going to simplify the intersection problem to just

In [3]:
function intersection(g, α, β)
    return 100 * β * α
end

intersection (generic function with 1 method)

Now we can get a starting benchmark for execution by generating some sample arrays:

In [4]:
geodesics = map(
    _ -> ones(Float64, (3, 300)),
    1:1000
)
geodesics[1][:, 1]

3-element Vector{Float64}:
 1.0
 1.0
 1.0

for which the above function takes

In [5]:
@btime renderdisk(π/100, 0.0, $geodesics)

  14.567 ms (2162 allocations: 35.68 MiB)


837741.0

## Naive GPU
CUDA.jl implements an `AbstractArray` type `CuArray`, which is the basis for interchanging memory with the GPU and the host machine.

The place to then start is by uploading all of the `Array` types into `CuArrays`, and then change the return init function `rinit` passed to `renderdisk` to be `CUDA.zeros`.

In [6]:
function upload(a::AbstractArray{<:Array})
    # upload
    [CUDA.CuArray(i) for i in a]
end

gpugeo = upload(geodesics)
typeof(gpugeo)

Vector{CuArray{Float64, 2}} (alias for Array{CuArray{Float64, 2}, 1})

Thus we get the naive benchmark:

In [7]:
function renderdiskgpu(α, β, gpugeo)
    renderdisk(α, β, gpugeo; rinit=CUDA.zeros)
end

@btime renderdiskgpu(π/100, 0.0, $gpugeo)

  253.622 ms (1154653 allocations: 64.41 MiB)


837741.0

which is significantly slower. This is to be expected, we are taking zero advantage of the GPU's hardware, and instead just running sloppy single threaded CPU code on the GPU.

## Using broadcasts
The GPU is very adapted to doing *embarassingly parallel* problems, in which very little scalar multiplication happens. The first thing we ought to do is assess where our GPU code is, and what computations we are exercising.

In the `renderdisk` function, we loop over `x` and `y` indices of a large zero array, representing the output image.

Each loop is independent, so lets try and isolate this stage and speed it up. To make the computation costly, we'll compute the quantities `β2` and `r`.

The approach when writing this is to remember
- each loop iteration is slow, but we can do very many simultaneously
- write to maximize number of unnested loops

In [8]:
function imager(rinit::Function; height=720, width=1080)
    data = rinit(Float64, (height, width, 3))
    
    h_mid = height ÷ 2
    w_mid = width ÷ 2
        
    values = map(
        (i) -> begin 
            # find position
            z = (i-1) ÷ (width * height)
            y = (i-1 - z*width*height) ÷ width
            x = i - ( z*width*height + y * width ) - w_mid
            y += 1
            z += 1
            
            β2 = atan(y - h_mid, x) # atan2
            r = sqrt(x^2 + (y - h_mid)^2)
            
            β2
        end,
        1:length(data)
    )
    
    sum(values)
end

@btime imager(zeros)

  51.198 ms (4 allocations: 35.60 MiB)


10166.387038500514

In [9]:
@btime imager(CUDA.zeros)

  49.822 ms (15 allocations: 17.80 MiB)


10166.387038500514

But we haven't yet taken advantage of the CUDA api. For this, we want to call the CUDA variant of `map`, which takes a CuArray as the second argument. Since we are just counting incrementally for `i`, we can pre-assemble this information using something like
```julia
reshape(collect(Float64, 1:width*height*3), (height, width, 3))
```

Implementing this new version, modifying `rinit` to be a `::Type`:

In [10]:
function imager(rinit::Type; height=720, width=1080)
    data = reshape(
        rinit(collect(Float64, 1:width*height*3)),
        (height, width, 3)
    )
    
    h_mid = height ÷ 2
    w_mid = width ÷ 2
        
    values = map(
        (i) -> begin 
            # find position
            z = (i-1) ÷ (width * height)
            y = (i-1 - z*width*height) ÷ width
            x = i - ( z*width*height + y * width ) - w_mid
            y += 1
            z += 1
            
            β2 = atan(y - h_mid, x) # atan2
            r = sqrt(x^2 + (y - h_mid)^2)
            
            β2
        end,
        data
    )
    
    sum(values)
end

imager (generic function with 2 methods)

In [11]:
@btime imager(Array)

  113.724 ms (8 allocations: 53.39 MiB)


10166.387038500514

In [12]:
@btime imager(CuArray)

  6.447 ms (4632 allocations: 17.87 MiB)


-5.789977393569645e6

That's looking more like it! However in doing this we have massively slowed down the CPU version over the initial implementation. But this is to be expected, and we can ammend the dispatch issues for CPU vs GPU later once we have squeezed performance.

*NB*: there is a slight biasing in this implementation towards the GPU, since for the CPU we call
```julia
Array(collect(...))
```
This barely changes the speed of the CPU version (<10 ms), but does reduce allocations significantly. I implemented it as above for ease of interoperability, though really I should be using the multiple dispatch to achieve this.

*Also NB*: we get GPU warnings about `atan` in `Base`. I'll investigate this later.

## Reimplementing
Let's reimplement the rendering function with just this change. I'm also going to take the liberty to slightly adjust the interface for multiple dispatch:

In [13]:
function renderdisk!(
    α::Float64,
    β::Float64,
    geodesics,
    data # no type to support recompilation on CuArray
    ;
    height::Int=720,
    width::Int=1080,
    fov_index::Int=200
    )
    
    
    h_mid = height ÷ 2
    w_mid = width ÷ 2
    
    n = size(geodesics)[1] ÷ 3
        
    values = map(
        (i) -> begin 
            # find position
            z = (i-1) ÷ (width * height)
            y = (i-1 - z*width*height) ÷ width
            x = i - ( z*width*height + y * width ) - w_mid
            y += 1
            z += 1
            
            β2 = atan(y - h_mid, x) # atan2
            r = sqrt(x^2 + (y - h_mid)^2)
            
            # additional steps here
            index = convert(Int, floor(1 + r / fov_index * n))
            
            # we no longer assign directly, but return new value
            ret = 0.0
            if index < n
                # has to calculate intersection three times, but this will be optimized later
                ret = truncator(intersection(geodesics[index:index+2, :], α, β + β2))
            end
            ret
        end,
        data
    )
    
    sum(values)
end

renderdisk! (generic function with 1 method)

Let's now benchmark this new implementation, noting that, for comparison purposes, we also need to time the array creation:

In [14]:
function test(geodesics, rinit::Type; height=720, width=1080)
    data = reshape(
        rinit(collect(Float64, 1:width*height*3)),
        (height, width, 3)
    )
    
    # squash so geodesics can be copied in CUDA.map call
    geodesics = vcat(geodesics...)
    
    renderdisk!(π/100, 0.0, geodesics, data; height=height, width=width)
end

@btime test($geodesics, Array)

  518.376 ms (376126 allocations: 2.64 GiB)


837741.0

In [15]:
@btime test($gpugeo, CuArray)

LoadError: InvalidIRError: compiling kernel broadcast_kernel(CUDA.CuKernelContext, CuDeviceArray{Float64, 3, 1}, Base.Broadcast.Broadcasted{Nothing, Tuple{Base.OneTo{Int64}, Base.OneTo{Int64}, Base.OneTo{Int64}}, var"#11#12"{Int64, Int64, Int64, Float64, Float64, CuDeviceMatrix{Float64, 1}, Int64, Int64, Int64}, Tuple{Base.Broadcast.Extruded{CuDeviceArray{Float64, 3, 1}, Tuple{Bool, Bool, Bool}, Tuple{Int64, Int64, Int64}}}}, Int64) resulted in invalid LLVM IR
Reason: unsupported dynamic function invocation (call to print_to_string(xs...) in Base at strings/io.jl:124)
Stacktrace:
  [1] [0m[1mstring[22m
[90m    @ [39m[90m./strings/[39m[90;4mio.jl:174[0m
  [2] [0m[1mthrow_checksize_error[22m
[90m    @ [39m[90m./[39m[90;4mmultidimensional.jl:881[0m
  [3] [0m[1m_unsafe_getindex[22m
[90m    @ [39m[90m./[39m[90;4mmultidimensional.jl:845[0m
  [4] [0m[1m_getindex[22m
[90m    @ [39m[90m./[39m[90;4mmultidimensional.jl:832[0m
  [5] [0m[1mgetindex[22m
[90m  

This is a bit problematic. I obviously don't want a unique copy of `gpugeo` per GPU thread, and would like to put these arrays in read-only memory, so they can be accessed as needed for computational purposes. The alternative would be to pre-assemble the data required for each pixel in-place, but that would conflate the memory needs massively.

## Texture Memory
I then stumbled accross an interesting section of the docs on [Texture Memory](https://juliagpu.github.io/CUDA.jl/stable/lib/driver/#CUDA.CuTexture-Tuple{Any}), which is interpolated read-only memory, nested around a CuArray, which I may be able to distribute over the threads:
> Construct a N-dimensional texture object with elements of type T as stored in parent.
>
>Several keyword arguments alter the behavior of texture objects:
>
>    - address_mode (wrap, clamp, mirror): how out-of-bounds values are accessed. Can be specified as a value for all dimensions, or as a tuple of N entries.
>
>    - interpolation (nearest neighbour, linear, bilinear): how non-integral indices are fetched. Nearest-neighbour fetches a single value, others interpolate between multiple.
>
>    - normalized_coordinates (true, false): whether indices are expected to fall in the normalized [0:1) range.

In [16]:
CuTextureArray(gpugeo[1])

LoadError: ArgumentError: CUDA does not support texture arrays for element type Float64.

But annoyingly we can't get Float64 support. We could cast it to Float32, which it does support, but I am increasingly unsure if these structures are what I want, amplified by the fact that the API is still experimental.

## Memmory Management
Examining the docs a little further, under their section on [Memmory Management](https://juliagpu.github.io/CUDA.jl/stable/lib/driver/#Memory-Management) are a few ways we can allocate and free memory on the GPU.

From the docs, there are two functions which I think might solve my problem:

> ```julia
> Mem.alloc(DeviceBuffer, bytesize::Integer)
> ```
>
> Allocate bytesize bytes of memory on the device. This memory is only accessible on the GPU, and requires explicit calls to `unsafe_copyto!`, which wraps `cuMemcpy`, for access on the CPU.

and possibly

> ```julia
> Mem.register(HostBuffer, ptr::Ptr, bytesize::Integer, [flags])
> ```
> 
> Page-lock the host memory pointed to by `ptr`. Subsequent transfers to and from devices will be faster, and can be executed asynchronously. If the `HOSTREGISTER_DEVICEMAP` flag is specified, the buffer will also be accessible directly from the GPU. These accesses are direct, and go through the PCI bus. If the `HOSTREGISTER_PORTABLE` flag is specified, any CUDA context can access the memory.

The second method would prevent having to upload, but would bottle neck at during reads. That being said, there are generally more pixels where no read is required, but I still am opting in favour of `alloc`.

I'm thinking it's going to give me the speed, and I can use it without too much restriction, since I'm allocating a very small amount of memory. I think. Let's check:

In [17]:
CUDA.total_memory()

4237033472

In [18]:
sizeof(Float64) * sum(length, geodesics)

7200000

Yeah, that's shouldn't be a problem, with plently of space to scale vertically!

### A single geodesic
Let's see if we can allocate a single curve, and access it in parallel:

In [19]:
curve = geodesics[1]
curve_p = CUDA.Mem.alloc(
    CUDA.Mem.DeviceBuffer,
    sizeof(Float64) * length(curve)
)

DeviceBuffer(7.031 KiB at 0x0000000b01c58c00)

Going to make sure I understood the API for freeing also:

In [20]:
CUDA.Mem.free(curve_p)

Yep, that works fine. 

Whilst trying to find more info on the `DeviceBuffer` API, I stumbled on a [few macros](https://juliagpu.github.io/CUDA.jl/stable/api/kernel/#Memory-types) which may also do what I want:

> ```julia
> @cuStaticSharedMem(T::Type, dims) -> CuDeviceArray{T,AS.Shared}
> ```
> 
> Get an array of type `T` and dimensions `dims` (either an integer length or tuple shape) pointing to a statically-allocated piece of shared memory. The type should be statically inferable and the dimensions should be constant, or an error will be thrown and the generator function will be called dynamically.

Which, if I understand correctly should allow me to do:

In [21]:
curve_mem = @cuStaticSharedMem Float64 (3, 300)

3×300 device array at Core.LLVMPtr{Float64, 3}(0x00007fa31539b1e0)

Nice, it returns a proper `LLVMPtr`! Now we're in familiar territory. 

*NB*: I'm not entirely certain where this memory is located however... I get the suspicion this has been allocated on the host and shared with the device, but I am unsure how to check. 

Thankfully I'm not the first person to ponder this, and found [this NVIDIA developer page](https://developer.nvidia.com/blog/using-shared-memory-cuda-cc/) containing more information on what "Shared Memory" means in the context of GPUs.

It is memory *allocated* on the GPU device, *shared* between threads.

In [22]:
isbits(curve_mem)

true

Perfect! So let's try this proof of concept with a conceptual function:

In [23]:
function shared_test(curve)
    
    curve_mem = @cuStaticSharedMem Float64 (3, 300)
    curve_mem[:] = curve[:]
    
    data = CuArray(collect(Float32, 1:10000)) # something decently sized
    map(
        i -> begin
            index = convert(Int, i % (3*300) + 1)
            curve_mem[index]
        end,
        data
    )
end

shared_test (generic function with 1 method)

In [24]:
# shared_test(curve)

This doesn't seem to work. I am encountering an `ERROR 700 ILLEGAL MEMORY ACCESS` which isn't boding well, and likewise I seem to be reading that shared memory is [significantly smaller](https://stackoverflow.com/a/20909276) than I'd at first hoped.

Also, shared memory is access via kernels, not by the CPU.